# Word2Vec Application on creating a Chatbot using Gensim

In [1]:
import json
json_file = 'intents.json'
with open('intents.json','r') as f:
    data = json.load(f)

data

[{'tag': 'welcome',
  'patterns': ['Hi',
   'How are you',
   'Is any one to talk?',
   'Hello',
   'hi are you available'],
  'responses': ['Hello, thanks for contacting us',
   'Good to see you here',
   ' Hi there, how may I assist you?']},
 {'tag': 'goodbye',
  'patterns': ['Bye', 'See you later', 'Goodbye', 'I will come back soon'],
  'responses': ['See you later, thanks for visiting',
   'have a great day ahead',
   'Wish you Come back again soon.']},
 {'tag': 'thankful',
  'patterns': ['Thanks for helping me',
   'Thank your guidance',
   "That's helpful and kind from you"],
  'responses': ['Happy to help!',
   'Any time!',
   'My pleasure',
   'It is my duty to help you']},
 {'tag': 'hoursopening',
  'patterns': ['What hours are you open?',
   'Tell your opening time?',
   'When are you open?',
   'Just your timing please'],
  'responses': ["We're open every day 8am-7pm",
   'Our office hours are 8am-7pm every day',
   'We open office at 8 am and close at 7 pm']},
 {'tag': 'pay

In [2]:
import pandas as pd
df = pd.DataFrame(data)

df

,tag,patterns,responses
0,welcome,"[Hi, How are you, Is any one to talk?, Hello, ...","[Hello, thanks for contacting us, Good to see ..."
1,goodbye,"[Bye, See you later, Goodbye, I will come back...","[See you later, thanks for visiting, have a gr..."
2,thankful,"[Thanks for helping me, Thank your guidance, T...","[Happy to help!, Any time!, My pleasure, It is..."
3,hoursopening,"[What hours are you open?, Tell your opening t...","[We're open every day 8am-7pm, Our office hour..."
4,payments,"[Can I pay using credit card?, Can I pay usin...","[We accept VISA, Mastercard and credit card, W..."


In [3]:
df['patterns'] = df['patterns'].apply(', '.join)
df

,tag,patterns,responses
0,welcome,"Hi, How are you, Is any one to talk?, Hello, h...","[Hello, thanks for contacting us, Good to see ..."
1,goodbye,"Bye, See you later, Goodbye, I will come back ...","[See you later, thanks for visiting, have a gr..."
2,thankful,"Thanks for helping me, Thank your guidance, Th...","[Happy to help!, Any time!, My pleasure, It is..."
3,hoursopening,"What hours are you open?, Tell your opening ti...","[We're open every day 8am-7pm, Our office hour..."
4,payments,"Can I pay using credit card?, Can I pay using...","[We accept VISA, Mastercard and credit card, W..."


In [4]:
from nltk.corpus import stopwords
from textblob import Word
stop = stopwords.words('english')
df['patterns'] = df['patterns'].apply(lambda x:' '.join(x.lower() for x in x.split()))
# df['patterns']= df['patterns'].apply(lambda x: ' '.join(x for x in x.split() if x not in string.punctuation))
df['patterns']= df['patterns'].str.replace("[^\w\s]",'')
df['patterns']= df['patterns'].apply(lambda x: ' '.join(x for x in x.split() if  not x.isdigit()))
df['patterns'] = df['patterns'].apply(lambda x:' '.join(x for x in x.split() if not x in stop))
df['patterns'] = df['patterns'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df

,tag,patterns,responses
0,welcome,hi one talk hello hi available,"[Hello, thanks for contacting us, Good to see ..."
1,goodbye,bye see later goodbye come back soon,"[See you later, thanks for visiting, have a gr..."
2,thankful,thanks helping thank guidance thats helpful kind,"[Happy to help!, Any time!, My pleasure, It is..."
3,hoursopening,hour open tell opening time open timing please,"[We're open every day 8am-7pm, Our office hour..."
4,payments,pay using credit card pay using mastercard pay...,"[We accept VISA, Mastercard and credit card, W..."


In [5]:
from gensim.models import Word2Vec

In [6]:
Bigger_list=[]
for i in df['patterns']:
    li = list(i.split(" "))
    Bigger_list.append(li)
model= Word2Vec(Bigger_list,min_count=1,size=300,workers=4)

In [7]:
# model.save("word2vec.model")
model.save("model.bin")

In [8]:
loaded_model = Word2Vec.load('model.bin')

In [9]:
vocab = list(loaded_model.wv.vocab)
vocab

['hi',
 'one',
 'talk',
 'hello',
 'available',
 'bye',
 'see',
 'later',
 'goodbye',
 'come',
 'back',
 'soon',
 'thanks',
 'helping',
 'thank',
 'guidance',
 'thats',
 'helpful',
 'kind',
 'hour',
 'open',
 'tell',
 'opening',
 'time',
 'timing',
 'please',
 'pay',
 'using',
 'credit',
 'card',
 'mastercard',
 'cash']

In [10]:
similar_words = model.most_similar('thanks')
print(similar_words)

[('helpful', 0.13922442495822906), ('hour', 0.0915229320526123), ('one', 0.07174398005008698), ('opening', 0.06882905960083008), ('time', 0.06164359301328659), ('talk', 0.05716725438833237), ('please', 0.055418722331523895), ('hi', 0.05442368984222412), ('tell', 0.04874475672841072), ('using', 0.04778452590107918)]


<ipython-input-10-c8e25aad01c6>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  similar_words = model.most_similar('thanks')


In [11]:
similarity_two_words = model.similarity('please','see')
print("Please provide the similarity between these two words:")
print(similarity_two_words)

Please provide the similarity between these two words:
0.0799671


<ipython-input-11-27dad22e16f5>:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  similarity_two_words = model.similarity('please','see')


In [12]:
similar = model.similar_by_word('kind')
print(similar)

[('bye', 0.09153793752193451), ('soon', 0.08413970470428467), ('helping', 0.05958578363060951), ('helpful', 0.058409251272678375), ('tell', 0.057453423738479614), ('talk', 0.04961136728525162), ('see', 0.03525525704026222), ('mastercard', 0.026565486565232277), ('hello', 0.020605210214853287), ('cash', 0.016043081879615784)]


<ipython-input-12-9e21e225c8bc>:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  similar = model.similar_by_word('kind')


## Homework
- Use USE (Universal Sentence Encoder) for Sentence Similarity